In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" "trl<0.9.0" peft accelerate bitsandbytes scikit-learn scipy joblib threadpoolctl
!pip install --no-deps --upgrade "flash-attn>=2.6.3" einops

  Cloning https://github.com/unslothai/unsloth.git to c:\users\sequi\appdata\local\temp\pip-install-24do95tq\unsloth_b1dc5fdfa9f840f7a8c096987a1f8ba3
  Resolved https://github.com/unslothai/unsloth.git to commit d0ca3497eb5911483339be025e9924cf73280178
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git 'C:\Users\sequi\AppData\Local\Temp\pip-install-24do95tq\unsloth_b1dc5fdfa9f840f7a8c096987a1f8ba3'


In [3]:
from unsloth import FastLanguageModel
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b-bnb-4bit",
    max_seq_length = 8192,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

ModuleNotFoundError: No module named 'triton'

In [ ]:
question = "Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q."
choices = "[ '0', '4', '2', '6' ]"

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Select the choice from the given 'Choices'.
    It's Very Important to have an output containing THE CHOICE !!!!
    ### Instruction:
    {}

    ### Choices:
    {}

    ### Response:
    {}"""

input_ids = tokenizer([
        alpaca_prompt.format(
            question, # instruction
            choices,
            "", # output - leave this blank for generation!
        )
    ], return_tensors="pt").to('cuda')
output_ids = model.generate(
    **input_ids,
    max_length=512,
    max_new_tokens=48,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    do_sample=False,
    num_beams=1,
    early_stopping=True,
    use_cache=False,
    repetition_penalty=3,
    length_penalty=-0.5
)
# Decode and print the output
output_text = tokenizer.batch_decode(output_ids)[0].strip()